## Imports

In [1]:
import os

parent_path = os.path.abspath("./../")
if os.path.basename(parent_path) == 'vdp':
    os.chdir(parent_path)

from glob import glob
import vdp
import re
import numpy as np


from vdp.vocabulary import FOSort, FOFunction
from vdp.fomodel import FOElement, FOModel
from vdp.exceptions import PartialInterpretationException




## Generate config.json files

In [63]:
# relative to
sgconfig = {
 'output_dir': './../data/sg_processed',
 'glove_path': './../../checkpoints/glove',
 'model_path': './../../checkpoints/model',
 'sg_tools_rel_path': './tools/relation_test_net.py',
 'sg_config_path': './configs/e2e_relation_X_101_32_8_FPN_1x.yaml',
 'cuda_device_port': 0,
 'n_proc': 1
}

foconfig = {
 "input_dir" : "./data/sg_processed",
 "output_dir" : "./data/fo_models",
 "rel_topk" : 20,
 "box_topk" : 20
}


In [64]:

vdp_puzzles =  glob("./data/raw/*")
for folder in vdp_puzzles:
    train = glob(folder + "/train*")
    test  = glob(folder + "/test*")
    name = os.path.basename(folder)
    config = {
        'name' : name,
        'train' : list(train),
        'test' : list(test),
        'sg_config' : sgconfig,
        'fo_config' : foconfig
    }
    json.dump(config, open(f"./config/{name}.json", 'w'))
    

Random stuff

In [2]:
all_puzzles_paths = glob("./vdp_smt/*/*")
all_puzzles_paths[0]

'./vdp_smt/2on1/2on1_ace-b123'

In [4]:
# def read_normalized_input(train_path, test_path):
#     train_test_predicts = []
#     for file_path in [train_path, test_path]:
#         lines = []
#         predicts = {}
#         with open(file_path, 'r') as f:
#             lines = [line.replace("\n", "") for line in f.readlines()]

#         mask = [("Image Path" in line) for line in lines]
#         images = [get_name.match(line).group(1) for idx, line in enumerate(lines) if mask[idx] if "Predicted" in line]
#         labels = [(np.where(mask[:idx+1])[0][-1], line) for idx, line in enumerate(lines) if not mask[idx] ]
#         for i in range(len(lines)):
#             if mask[i] and "Predicted" in lines[i]:
#                 img_name = get_name.match(lines[i]).group(1)
#                 next_i = i + np.argmax(mask[i+1:])
#                 predicts[img_name] = list(zip(lines[i+1:next_i:2], lines[i+2:next_i:2]))
        
#         train_test_predicts.append(predicts)
        
#     return {**train_test_predicts[0], **train_test_predicts[1]}


# # for puzzle in all_puzzles_paths:
# #     train_path = puzzle + "/" + os.path.basename(puzzle) + "_train_out.txt"
# #     test_path = puzzle + "/" + os.path.basename(puzzle) + "_test_out.txt"
# #     yolo_output = (read_normalized_input(train_path, test_path))
# #     for img_name, img in yolo_output.items():
# #         img_params = [(*item[0].split(": "), *split_bb.findall(item[1].split(": ")[-1])) for item in img ]
# #         print(img_name, img)
# #         print(img_params)
# #         print()
    
# # #     break

file_path = './vdp_smt/2on1/2on1_ace-b123/2on1_ace-b123_train_out.txt'


extract_img = re.compile(r"Enter Image Path: .+\/([^/]+).jpg: Predicted in \d*\.?\d* seconds.")
extract_bb = re.compile(r"Bounding Box: Left=(\d+), Top=(\d+), Right=(\d+), Bottom=(\d+)")

process_img = lambda line: extract_img.search(line).groups()
process_obj = lambda line: line.replace("%", "").split(": ")
process_bb  = lambda line: extract_bb.search(line).groups()


In [74]:
rx_dict = {
    'start' : re.compile(r"Enter Image Path: .+\/(?P<name>[^/]+).jpg: Predicted in \d*\.?\d* seconds.\n"),
    'obj' : re.compile(r"(?P<obj>\w+): (?P<score>\d+)%\n"),
    'bb' : re.compile(r"Bounding Box: Left=(?P<left>\d+), Top=(?P<top>\d+), Right=(?P<right>\d+), Bottom=(?P<bottom>\d+)")
}


def _parse_line(line):
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return (key, match)
    return (None, None)


def _parse_file(file_path):
    data = list()
    
    with open(file_path, 'r') as fp:
        line = fp.readline()
        while 'Predicted' in line:
            key, match = _parse_line(line)
            name = match.group('name')
            img_line = fp.readline()
            img_data = dict()
            objs = list()
            while img_line and 'Predicted' not in img_line:       
                key, match = _parse_line(img_line)
                if key == 'obj':
                    objs.append((match.group('obj'), float(match.group('score'))))

                if key == 'bb':
                    bb = (match.group('left'), match.group('top'), match.group('bottom'), match.group('right'))
                    img_data = {**img_data, **{obj : {'score' : score, 'bb' : bb} for obj, score in objs}}
                    objs = list()
                
                img_line = fp.readline()
            
            data.append((name, img_data))
            line = img_line
    return data





def parse(yolo_folders, output_folder):
    for folder_path in yolo_folders:
        print(folder_path)
        name = os.path.basename(folder_path)
        output_dir =f"{output_folder}/{name}" 
        print(output_dir)
#         os.makedirs(output_dir, exist_ok=True)
        train_path = folder_path + "/" + name + "_train_out.txt"
        test_path = folder_path + "/" + name + "_test_out.txt"
        data = {name : [*_parse_file(train_path), *_parse_file(test_path)]}
        break

    return data

data = parse(glob("./vdp_smt/*/*"), './data/yolo_processed')



./vdp_smt/2on1/2on1_ace-b123
./data/yolo_processed/2on1_ace-b123


In [86]:
centroid = lambda bb : ((int(bb[0]) + int(bb[2])) / 2, (int(bb[1]) + int(bb[3])) / 2)


def relate(c1, c2): # where is c1 w.r.t c2
    x1, y1 = c1
    x2, y2 = c2
    
    return ["right" if x1 >= x2 else "left", "above" if y1 >= y2 else "below"]

output_dir = "./data/yolo_fo_models"
key = '2on1_ace-b123'
val = data[key]

# for img in val:
#     folder = 
    
for name, data in val:
    batch = name.split("_")[0]
    os.makedirs(f"{output_dir}/{key}/{batch}", exist_ok=True)
    output_json = f"{output_dir}/{key}/{batch}/{name}.json"
    
    img_data = data
    
    

In [119]:


centroid = lambda bb : ((int(bb[0]) + int(bb[2])) / 2, (int(bb[1]) + int(bb[3])) / 2)
dist = lambda c1, c2 : ((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2)**0.5

def relate(c1, c2): # where is c1 w.r.t c2
    x1, y1 = c1
    x2, y2 = c2
    
    return ["right" if x1 >= x2 else "left", "above" if y1 >= y2 else "below"]




labels = [obj + "_label" for obj in img_data.keys()]

rel_labels = list()
for obj1, obj_data1 in img_data.items():
    for obj2, obj_data2 in img_data.items():
        if obj1 != obj2:
            c1 = centroid(obj_data1['bb'])
            c2 = centroid(obj_data2['bb'])
            rel_labels.append(['obj', obj1, 'obj', obj2, "_".join(relate(c1, c2)), obj_data1['score'] * obj_data2['score'] * 1e-4])



rel_labels

[['obj', 'sofa', 'obj', 'cat', 'left_above', 0.6272],
 ['obj', 'cat', 'obj', 'sofa', 'right_below', 0.6272]]

{'sorts': ['object', 'label', 'scores'],
 'predicates': {'left_above': '(object, object)',
  'right_below': '(object, object)',
  'has label': '(object, label)',
  'has score': '(object, object, score)'},
 'elements': {'object': ['obj_cat', 'obj_sofa'],
  'label': ['sofa', 'cat'],
  'scores': [0.6272, 0.6272]},
 'interpretation': {'left_above': [('obj_sofa', 'obj_cat')],
  'right_below': [('obj_cat', 'obj_sofa')],
  'has label': [('obj_sofa', 'sofa'), ('obj_cat', 'cat')],
  'has score': [('obj_sofa', 'obj_cat', 0.6272),
   ('obj_cat', 'obj_sofa', 0.6272)]},
 'raw': [['obj', 'sofa', 'obj', 'cat', 'left_above', 0.6272],
  ['obj', 'cat', 'obj', 'sofa', 'right_below', 0.6272]]}

In [7]:

relations = {}
for label1, pred1, bb1 in img_params:
    for label2, pred2, bb2 in img_params:
        if label1 != label2:
            c1 = centroid(bb1)
            c2 = centroid(bb2)
            relations[(label1, label2)] = relate(c1, c2)
        else:
            relations[(label1, label2)] = ['labelOf']
            
labels, preds, bbs = zip(*img_params)

relations


{('bed', 'bed'): ['labelOf'],
 ('bed', 'sofa'): ['right', 'above'],
 ('bed', 'cat'): ['left', 'above'],
 ('sofa', 'bed'): ['left', 'below'],
 ('sofa', 'sofa'): ['labelOf'],
 ('sofa', 'cat'): ['left', 'above'],
 ('cat', 'bed'): ['right', 'below'],
 ('cat', 'sofa'): ['right', 'below'],
 ('cat', 'cat'): ['labelOf']}

In [8]:
vdpobject = FOSort('Object')
vdplabel = FOSort('Label')
vdpbool = FOSort('Bool')


In [9]:
vdpconstants = dict()
for label in labels:
    if label not in vdpconstants:
        vdpconstants[label] = FOFunction(label, tuple([vdplabel]))
    
    
unique_labels = list(vdpconstants.keys()) # @TODO REPLACE!!!!


In [10]:
# on = FOFunction('On', (vdpobject, vdpobject, vdpbool))
relation_labels = ['left', 'right', 'above', 'below']
vdprelations = dict()

for rl in relation_labels:
    vdprelations[rl] = FOFunction(rl, (vdpobject, vdpobject, vdpbool))


vdprelations['labelOf'] = FOFunction('labelOf', (vdpobject, vdplabel, vdpbool))
vdprelations

{'left': <vdp.vocabulary.FOFunction at 0x10889af10>,
 'right': <vdp.vocabulary.FOFunction at 0x10889a250>,
 'above': <vdp.vocabulary.FOFunction at 0x10889ab90>,
 'below': <vdp.vocabulary.FOFunction at 0x10889a790>,
 'labelOf': <vdp.vocabulary.FOFunction at 0x10889ad10>}

In [11]:
vdpvocabulary = (set([vdpobject, vdplabel]), set(list(vdpconstants.values()) + list(vdprelations.values())))


In [12]:
vdpobjects = {label : FOElement(label + "_object", vdpobject) for label in labels}
vdplabels = {label : FOElement(label + "_label", vdplabel) for label in unique_labels}

vdpelements = {'Object': set(vdpobjects.values()), 'Label': set(vdplabels.values())}

In [13]:
labelconst_interpretation = {const : vdplabels[label] for label, const in zip(unique_labels, list(vdpconstants.values()))}
labelconst_interpretation

{<vdp.vocabulary.FOFunction at 0x10889a650>: <vdp.fomodel.FOElement at 0x111a026d0>,
 <vdp.vocabulary.FOFunction at 0x10889a2d0>: <vdp.fomodel.FOElement at 0x111a02810>,
 <vdp.vocabulary.FOFunction at 0x10889a510>: <vdp.fomodel.FOElement at 0x111a02910>}

In [14]:

    

rel_interpretation = dict()
# for rel_sym, rel in vdprelations.items():
#     if 'labelOf' in rel_sym:
#         rel_interpretation[rel] = {(obj, lab) : rel_sym in relations[(obj_sym, lab_sym)]  for obj_sym, obj in vdpobjects.items() for lab_sym, lab in vdplabels.items()}
#     else:
#         rel_interpretation[rel] = {(obj1, obj2) : rel_sym in relations[(obj1_sym, obj2_sym)]  for obj1_sym, obj1 in vdpobjects.items() for obj2_sym, obj2 in vdpobjects.items() if obj1_sym != obj2_sym}

        
rel_interpretation[vdprelations['labelOf']] = {(obj, lab) : 'labelOf' in relations[(obj_sym, lab_sym)]  for obj_sym, obj in vdpobjects.items() for lab_sym, lab in vdplabels.items()}


rel_interpretation

{<vdp.vocabulary.FOFunction at 0x10889ad10>: {(<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a026d0>): True,
  (<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a02810>): False,
  (<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a02910>): False,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a026d0>): False,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a02810>): True,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a02910>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a026d0>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a02810>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a02910>): True}}

In [15]:
vdpinterpretation = {**labelconst_interpretation, **rel_interpretation}


In [16]:
model = FOModel(vdpvocabulary, vdpelements, vdpinterpretation)


In [17]:
tst = FOElement('x', vdpobject)
model.interpret(term=[vdprelations['labelOf'], [tst], [vdpconstants['cat']]], interpretation_extension={tst: vdpobjects['cat']})

True